In [1]:
#last update: Oct 08 2024
#this file read in the selected sampling data from sampler
#and output the txt file for median data for given sampling trails
#read in data
#process the data to data point by statiscal method like find the medium
#calculate the error bar
#write teh data, error bar and time info into a txt file for plotting



import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import sys
import os
import inspect
from datetime import datetime, timezone, timedelta
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#sys.path.append(os.environ['BEACON_INSTALL_DIR'])
sys.path.append('home/youwei/project_beacon/2024_10')
#from beacon_data_reader import Reader 
#from beaconroot.examples.beacon_data_reader import Reader 
from Reader_class import Reader 
import ROOT


In [16]:
#generate an array for the date we what to collect data for:
# Generate an array of filename
days_apart=15
start_date = datetime(2024, 2, 14, 0, 0, 1)
end_date = datetime(2024, 5, 23, 0, 0, 1)
data_days_arr=[]
test_date=start_date
output_filepath=[]
sample_directory = "sampling/"
while test_date<end_date:
    data_days_arr.append(test_date)
    date_str = test_date.strftime("%Y-%m-%d")
    sample_filename="sampling_at_"+date_str
    output_filepath.append(os.path.join(sample_directory, sample_filename))
    test_date = test_date + timedelta(days=days_apart)


print(output_filepath)

['sampling/sampling_at_2024-02-14', 'sampling/sampling_at_2024-02-29', 'sampling/sampling_at_2024-03-15', 'sampling/sampling_at_2024-03-30', 'sampling/sampling_at_2024-04-14', 'sampling/sampling_at_2024-04-29', 'sampling/sampling_at_2024-05-14']


In [17]:
# Initialize empty lists to store the data
run_array = []
readout_time_array = []
entry_array = []



class Event:
    def __init__(self, run, entry, channel, readout_time, t, waveform):
        self.run = run
        self.entry = entry
        self.channel = channel
        self.readout_time = readout_time
        self.waveform = waveform
        self.t=t
        self.datetime = datetime.fromtimestamp(self.readout_time)
        
    def set_fft(self):
        # Compute the FFT of the waveform
        waveform_fft = np.fft.fft(waveform)
        
        # Compute the corresponding frequency bins
        dt = (self.t[1] - self.t[0])  # Time step between consecutive time points
        freqs = np.fft.fftfreq(len(waveform), dt)
        #print(len(waveform))
        
        # Only plot the positive frequencies and the corresponding FFT values
        self.freqs = freqs[:len(freqs) // 2]
        self.fft = np.abs(waveform_fft[:len(waveform_fft) // 2])

    def plot_fft(self):
        plt.subplot(2, 1, 2)
        plt.plot(self.freqs, self.fft)
        plt.title(f"Run {self.run}, Event {self.entry}, Channel {self.channel} - Frequency Domain (FFT)")
        plt.ylabel("Amplitude")
        plt.xlabel("Frequency (GHz)")
        plt.tight_layout()
        plt.show()

    def plot_wf(self):
        plt.subplot(2, 1, 2)
        plt.plot(self.t, self.waveform)
        plt.title(f"Run {self.run}, Event {self.entry}, Channel {self.channel} - Time Domain")
        plt.ylabel("Amplitude")
        plt.xlabel("time")
        plt.tight_layout()
        plt.show()
    
    def set_V_rms(self, vrms=None):
        if vrms is None:
            # Calculate V_rms from waveform
            waveform = self.waveform
            self.mean = np.mean(waveform)
            
            # Calculate the squared differences from the mean
            squared_diffs = (waveform - self.mean) ** 2
            
            # Calculate the variance (mean of squared differences)
            variance = np.mean(squared_diffs)
            
            # Calculate the standard deviation (square root of variance)
            self.V_rms = np.sqrt(variance)
        else:
            # Set V_rms directly
            self.V_rms = vrms

        return self.V_rms

def find_median_with_error(Event_per_point_array):
    # Extract the Vrms values from the events
    vrms_array = [event.set_V_rms() for event in Event_per_point_array]
    
    # Calculate the median
    vrms_median = np.median(vrms_array)
    
    # Number of bootstrap samples
    n_bootstrap = 1000
    
    # Array to hold bootstrap medians
    bootstrap_medians = np.zeros(n_bootstrap)
    
    # Bootstrap resampling
    for i in range(n_bootstrap):
        # Resample with replacement
        sample = np.random.choice(vrms_array, size=len(vrms_array), replace=True)
        # Compute the median of the resampled data
        bootstrap_medians[i] = np.median(sample)
    
    # Calculate the 1 sigma (68% confidence interval) error bars
    lower_bound = np.percentile(bootstrap_medians, 16)
    upper_bound = np.percentile(bootstrap_medians, 84)
    vrms_median_20 = np.percentile(bootstrap_medians, 20)
    # Calculate the error bar
    error_bar = (vrms_median - lower_bound, upper_bound - vrms_median)

    t=(Event_per_point_array[len(Event_per_point_array)//2]).readout_time
#t? use middle value?
# how to find the midian error
    #std(median)=1.2533*std(mean)
    return vrms_median, vrms_median_20, t, error_bar



Run Array: [702, 702, 702, 702, 702, 702, 702, 702, 702, 702]
Readout Time Array: [1707890365.0, 1707890399.0, 1707890519.0, 1707890327.0, 1707890411.0, 1707890445.0, 1707890326.0, 1707890489.0, 1707890508.0, 1707890494.0]
Entry Array: [9422, 9490, 9730, 9347, 9515, 9583, 9345, 9671, 9707, 9680]


In [ ]:
# Open the file and read the data
with open(output_filepath[args.param1], "r") as file:
    current_array = None  # This will keep track of which array we are currently reading into
    
    for line in file:
        # Remove any leading/trailing whitespace
        line = line.strip()

        # Check if the line indicates the start of a new array
        if line.startswith("Run Array:"):
            current_array = "run"
        elif line.startswith("Readout Time Array (timestamps):"):
            current_array = "readout_time"
        elif line.startswith("Entry Array:"):
            current_array = "entry"
        elif line:  # If the line is not empty, it contains data
            # Depending on which array we're reading, process the line
            if current_array == "run":
                run_array = [int(x.strip()) for x in line.split(",") if x.strip()]
            elif current_array == "readout_time":
                readout_time_array = [float(x.strip()) for x in line.split(",") if x.strip()]
            elif current_array == "entry":
                entry_array = [int(x.strip()) for x in line.split(",") if x.strip()]
                break

# Print the lists to verify the data
print("Run Array:", run_array[:10])
print("Readout Time Array:", readout_time_array[:10])
print("Entry Array:", entry_array[:10])

In [18]:
datapath = "/project/avieregg/beacon-flower8/root/"

# change this where ever you store data runs
events = []
channel =4
break_time = 600  #time between every data points


# Loop over each run and entry
for i in range(len(run_array)):
    # Initialize the reader for each run
    reader = Reader(datapath, run_array[i])
    
    # Set the specific entry for the current run
    reader.setEntry(entry_array[i])
    
    # Get the time array (assuming it's the same for all channels)
    t = reader.t()
    
    # Read the waveform for the current channel
    waveform = reader.wf(channel)
    
    # Create an Event object and add it to the events list
    event = Event(run_array[i], entry_array[i], channel, readout_time_array[i], t, waveform)
    event.set_V_rms()
    events.append(event)
    
events_slice=[]
samples=[]
for event_id in range(len(events)):
    end=False
    if event_id == len(events)-1:
        end=True
    else:
        t_diff = events[event_id+1].readout_time - events[event_id].readout_time
        
    if t_diff < break_time and not (end):
        samples.append(events[event_id])
    elif end :
        samples.append(events[event_id])
        events_slice.append(samples)
    else:
        events_slice.append(samples)
        samples=[]


vrms_median_arr=[]
vrms_20_arr=[]
t_array=[]
error_bar_arr=[]
for e_slice in events_slice:
    vrms_median, vrms_20, t, error_bar = find_median_with_error(e_slice)
    vrms_median_arr.append(vrms_median)
    vrms_20_arr.append(vrms_20)
    t_array.append(t)
    error_bar_arr.append(error_bar)
    # mid_n=len(e_slice)//2
    # event=Event(e_slice[mid_n].run, e_slice[mid_n].entry, channel, e_slice[mid_n].readout_time, e_slice[mid_n].t, e_slice[mid_n].waveform)
    # #vrms_mean=np.mean([sample.V_rms for sample in e_slice])
    # #event.set_V_rms(vrms_mean)
    # vrms_median=np.median([sample.V_rms for sample in e_slice])
    # event.set_V_rms(vrms_median)
    
    # plot_events.append(event)     

In [ ]:
# Extract datetime and V_rms into separate arrays
readoutts=t_array
datetimes = [datetime.fromtimestamp(sec) for sec in t_array]
#V_rms_values = vrms_median_arr
dates = mdates.date2num(datetimes)



# # Convert to NumPy arrays for calculations
# dates_np = np.array(dates)
# V_rms_np = np.array(vrms_median_arr)

# # Create the scatter plot
# plt.figure(figsize=(10, 6))
# plt.scatter(dates, V_rms_values, color='blue', marker='o')
# # plt.plot(dates_avg_hr, moving_avg_hr, color='green', linestyle='-', linewidth=2, label='hrly Moving Average')
# # plt.plot(dates_avg, moving_avg_3hr, color='cyan', linestyle='--', linewidth=2, label='3hr Moving Average')

# # Format the x-axis to display dates nicely
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H:%M'))
# plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

# # Rotate the date labels for better readability
# plt.gcf().autofmt_xdate()

# # Add labels and title
# plt.xlabel('Datetime')
# plt.ylabel('V_rms')
# plt.title(f"STD over the day for channel {channel}")

# # Calculate the minimum and median of V_rms
# min_value = np.min(V_rms_values)
# max_value = np.max(V_rms_values)
# median_value = np.median(V_rms_values)
# plt.ylim(min_value-0.5,max_value+0.5 )
# #plt.legend()


# # Create the filename with start datetime and channel number
# filename = f"STD_plot_{datetimes[0]}_channel_{channel}_random_median.png"

# # Save the figure
# plt.savefig(filename)

# # Show the plot
# plt.show()

filenametxt = f"STD_plot_{datetimes[0]}_channel_{channel}_random_median_with20.txt"
data_generated_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Open the file for writing
with open(filenametxt, 'w') as file:
# Write the filename string as the first line
    file.write(f"{filenametxt}\n")
    
    # Write the Readout Time array
    file.write("Readout Time:\n")
    file.write(",".join(str(t) for t in t_array) + '\n')
    
    # Write the V_rms median array
    file.write("\nV_rms Median:\n")
    file.write(",".join(str(vrms_median) for vrms_median in vrms_median_arr) + '\n')
    
    # Write the 20% V_rms value array
    file.write("\nV_rms at lowest 20%:\n")
    file.write(",".join(str(vrms_20) for vrms_20 in vrms_20_arr) + '\n')
    
    # Write the Error Bars
    file.write("\nError Bars (Lower, Upper):\n")
    for lower_err, upper_err in error_bar_arr:
        file.write(f"({lower_err}, {upper_err}), ")
    file.write("\n")


